In [7]:
import pandas as pd
import numpy as np
import re
import dataframe_image as dfi
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%matplotlib notebook

import statsmodels.api as sm 
import statsmodels.formula.api as sf

In [2]:
df = pd.read_csv('../data/bayut/cleaned/ready_flats_valuation.csv')

In [3]:
# Calculate first and third quartile
first_quartile = df['price'].describe()['25%']
third_quartile = df['price'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove outliers
data = df[(df['price'] > (first_quartile - 3 * iqr)) &
            (df['price'] < (third_quartile + 3 * iqr))]

In [11]:
# find set of parameters with maximum R-squared
feats = {'district', 'surface', 'beds', 'baths', 'furnishing', 'views'}

# iterate through all parameters
def brute_force(params: set, formula: str, result: set) -> str:
    if not params:
        result.add(formula.lstrip('+ '))
        return 
    else:
        new_params = params.copy()
        for el in params:
            new_params.remove(el)

            brute_force(new_params, formula + ' + ' + el, result)
            brute_force(new_params, formula, result)

res = set()          
brute_force(feats, '', res)
res.remove('')

# calculate Adjusted R-Square for each set of parameters
for formula in res:
    lm = sf.ols(formula='price ~ ' + formula, data=data).fit()
    print(f'{lm.rsquared_adj:.2}', formula)

0.45 surface + views + beds
0.42 baths + surface + beds
0.7 furnishing + district + beds
0.39 surface
0.69 baths + views + district + beds
0.43 surface + views
0.68 views + beds + district
0.64 baths + views + district
0.026 furnishing
0.29 baths
0.71 surface + views + beds + district
0.39 views + baths + beds
0.71 surface + beds + district
0.38 beds + views
0.42 furnishing + baths + surface
0.71 furnishing + views + surface + district
0.43 views + surface
0.65 furnishing + baths + district
0.69 baths + district + beds
0.71 views + baths + surface + district + beds
0.48 furnishing + district
0.63 baths + district
0.68 beds + district
0.46 views + district
0.69 baths + views + beds + district
0.45 baths + views + surface + beds
0.41 furnishing + surface
0.71 surface + district + beds
0.35 furnishing + views + baths
0.7 furnishing + baths + district + beds
0.68 beds + views + district
0.47 furnishing + views + baths + surface + beds
0.71 furnishing + views + baths + district + beds
0.7 b